# High Performance Computing at Dartmouth
*Written by Courtney Jiminez & Luke Chang*

## What is High Performance Computing?

https://www.youtube.com/watch?v=nIBu1EFYmBU 00:13 - 01:21

## HPCs at Dartmouth

https://rc.dartmouth.edu/index.php/discoveryhpc/

### Discovery Cluster

Where is Discovery located? In the basement of Baker Berry! In a huge room full of servers. Racks of computers. Each computer is a node - newer nodes have 32 cores with 8GB per core. Used by all Departments (PBS, CS, Physics & Astronomy, etc.)

### System Layout

Head Node: what you log-on to when you SSH in with your NetID. Can edit code, submit jobs to scheduler from head node - but DON'T want to run jobs from head node. If the head node computer resources get tied up in running jobs, it can keep others from logging in/opening or copying files, etc. 

CPU Node: can't directly log into these - can schedule jobs from head node that get executed on CPU nodes by the scheduler, but can't directly log on (unless interactive node). 

Storage?

### Research Computing Team

Centralized high-performance computing system group. 

services.dartmouth.edu - where help/request forms are for research computing (try searching for Discovery, SLURM, etc.) Locally written documentation for things that people have problems with. 

Simple, basic examples of what you can do with the scheduler. 

If you have a Discovery question, should go to services.dartmouth.edu, search for a key word, see if it's already written up. If you have that question, likely a bunch of others did as well, and RC has tried to answer it within the Dartmouth context.

research.computing@dartmouth.edu with questions.

https://services.dartmouth.edu

### DBIC Resources on Discovery

DBIC owns 15 nodes (16 cores?) on Discovery. Acess to ~1200 CPUs (Luke, what is this x5 multiplier? How do we go from 15x16=240 to 1200 CPUs?)

DBIC Node (Ndoli): high-RAM node (1.5 TB), 20-24 CPUs. Can only access it by directly logging in. UserID@ndoli.dartmouth.edu (not available on scheduler). 

### PBS HPCs

Dr. Zeus? LOL. Still availalbe - 2-3 nodes, 64 cores, 5-12 GB RAM per core (1.5 TB). Not *realy* maintained. 30-40 TB storage (likely completely full?) Primarily used by Social Area. 

Hydra - Haxby Lab.

## How do I access Discovery? 

### Need to be on Campus Network

Eduroam
OpenVPN

### SSH From Terminal

Can't use Key Pairs.

## How do I navigate Discovery? 

### Linux


### BASH


### Text Editors
VIM/Nano

### .bashrc .bash_profile

when you login.

don't load modules automatically. 

### ACL Permissions

ls -al

horribly confusing and annoying. 14 different permissions.

listacl -v foldername

POSIX permissions don't apply :( 

see AskPBS

know what they are when you run into a problem and google it LOL

can also just ask research computing to update your permissions

https://services.dartmouth.edu/TDClient/1806/Portal/KB/ArticleDet?ID=88459

## Where is my data stored on Discovery?

### Dart-FS File System

DBIC has some storage. 

Lab Storage: For the most part, each lab is responsible for buying their own storage on Discovery (~$120/TB/year)

Personal Storage: 50GB in Home Directory

### Scratch Space

LOCAL SCRATCH
local scratch space on every node of discovery (1 TB), much faster than network storage so if your job needs to keep any amount of state information/files it'd help if you wrote it on /scratch. can then copy things from /sratch to dart-fs at the end of your script - clean up any temp files in local scratch at end of script as well. Can make a big difference in some sorts of jobs. Worth noting that local scratch isn't managed - may not be a lot of room on it, but everything gets purged after ~45 days. Usually there's a good amount of space (not too many people use it), but a chance you could end up running a job on a node where the local scratch is a little full. 

GLOBAL SCRATCH 
We also have a GLOBAL SCRATCH dart-fs-hpc/scratch but it is not any faster than just accessing lab share on dart-fs etc. 

### Transferring Data to Discovery

Moving Data from the Command Line:

scp, rsync - don't do this from the head (login) node! run interactive job

datalad? dbic handbook

## How do I manage software on Discovery? 

### Modules

Modules are software that are already installed on Discovery (Python, R, FSL). Module commands allow you to modify your environment on Discovery. 

If you want more control over the software you are using, can always build your own modules in your home/lab directory.

You can load these software modules when you want to use these programs. Can do manually (module load command). 

Can see a list of modules by using 'module avail' command. 

module load
module avail
module list
module unload
module purge

weird* - if you've loaded a module, then submit a job, the batch job inherits the environment from which it was submitted - can avoid this by: 

using the #!/bin/bash -l flag in shebang line of sbatch job script. this sets up a log-in environment (reads bashrc profile)

putting module purge at the start of each job script and then load specific modules you need 

### Conda

A way to install additional Python/R packages to your computing environment on the cluster. Don't need any privilieges - installs dependencies, creates separate environments to avoid conflicts. 

Configured inside .bashrc profile - so if using with a SBATCH script, shebang line needs -l flag (#!/bin/sh -l). This results in the SBATCH script reading your .bashrc and .bashprofile scripts and behaving like your log-in shell. Without, .bashrc doens't get read and Conda isn't available. SBATCH script inherits current environment.

### Containers

A reproducible environment in which you control what software is in it. Can load container onto the cluster, run scripts that use software in that container. 

Docker is most popular container, but requires ROOT ACCESS to the computer it is being run on. Therefore, can't use Docker on Discovery.

Singularity - can be run on the cluster (doesn't require ROOT ACCESS). Can convert a Docker container to a Singularity container, etc. 

Can build them in home/lab directories.

YCRC Workshop

## How do I run jobs on Discovery?

### SLURM

SLURM is an open source, free and very flexible scheduler.

SLURM is the SCHEDULER for Discovery - it is in control of a whole bunch of computers. When we submit jobs, we request a certain number of CPUs, a certain amount of RAM, and specify how long the job will take. The scheduler decides which computer to run that job on. Potentially 1000s of people submitting jobs, and scheduler is trying to optimize resource allocation.  

### Common SLURM Commands

SBATCH - 

SRUN - 

SQUEUE - SLURM function that will list everything in the queue for your SLURM account (DBIC group). SQUEUE will show what is running, who owns it, and what is queued. Will also show WHY it's queued (e.g. group limit reached). 

SCANCEL - 

SACCTMGR - show associations, to find out what your scheduling limits are. 

SSTAT - 

SINFO - 

SACCT - 

SEFF - doesn't really work well for array jobs. 

### Requesting Resources

How do I know how many CPUs, how much RAM, and how much time I'll need for my job? 

If not enough CPUs, will take longer. If not enough RAM, job will error out. If you request TOO MANY resources, system will give you what you ask for (and you'll be charged $$) which may limit where your job can run and other's jobs from running. 

NEED TO SPECIFY: CPU count, Memory, Walltime. 
If you don't, the scheduler allocates 1 CPU, 4 GB RAM, and 1 hour. 

Limits for a given Resource Pool: 
CPU:
WALLTIME:

DBIC pool: allowed to use a certain number of CPUs concurrently, submit jobs for a number of total day's run time. If limit is hit, nobody in group can submit jobs until something finishes. 

Intended to keep one group from occupying entire cluster. 

If you submit a job (and haveb'nt reached a limit cap), scheduler should take about 30 seconds to decide where to put your job. If it doesn't start immediately, job goes into a queue. 

### Running Batch Jobs

Most jobs on clusters are batch jobs. Text files that get run as a script. 
Submit a script.

You submit a job by preparing a small shell script that has SBATCH directives. Script can be written in any shell you like but most examples use BASH - whatever you specify in shebang line (#!/bin/bash). SBATCH stops parsing script at first uncommented line.

NODES (never need > 1 node) -

TASKS/NODES (PROCESSES/NODE) - if your job is a call to one program (even if multi-threaded), it just does one task - only need 1 core here (which is the default). Never need more than 1 unless running an MPI job that knows how to run in parallel. 

CPUs/TASK (CORES/PROCESS) - can request more than 1 for multi-threaded/multi-process jobs. (e.g. fMRIprep can run multi-threaded, can specify 16 cpus/threads)

RAM - hardest thing to estimate. Ideally you run a similar job in an unrestricted system and use top to see how much memory it allocates for the job, then request similar amount.  

Walltime (doesn't hurt too much to overestimate but don't get wild LOL - will keep others' jobs from starting). Noticed your job isn't finished but walltime is running out? Can extend walltime (for up to 30 days!) but need to ask research computing to do this. Email research.computing@dartmouth.edu. You can use the SBATCH qnotify directive to let you know a certain amont of time before your job terminates.

Other SBATCH directives: check out this link. https://slurm.schedmd.com/sbatch.html

### Arrays: Running Multiple Batch Jobs at Once

Job Arrays - if you have a lot of jobs you want to run that are veyr similar but differ on some parameter given to a differnet program or perhaps a data input file, then you can submit one job script with a job array that specifies the different variable for each job. Will launch the amount of jobs specified by the number of different variables listed in the array. (SID example). Can throttle - say no more than 10 at a time (nice to others :), will get to them as earlier jobs finish).

### Running Interactive Jobs

Like a remote session - useful for development, debugging, or interactive coding environments (R, Python, Matlab). 

### Monitoring Jobs 

while it's running: 
launch job - use squeue to see where it's running - then ssh to that node and use top or htop to see resource use. 
scontrol

after it's finished: 
seff
sacct

### Scheduler Etiquette

How can I figure out who's running jobs on Discovery? SLURM scheduling command - squeue: lists all active jobs on the cluster.

User ID - dartmouth lookup

## Can I use Jupyter Notebook on Discovery?

AskPBS.org link

## Have Additional Discovery Questions?

### AskPBS


### DBIC Handbook
https://dbic-handbook.readthedocs.io/en/latest/discovery.html

## References

Knowledge Base

DBIC Handbook

YCRC YouTube